In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

#### Miscellaneous functions

In [2]:
def flatten(array): # when np.flatten() doesn't work
    temp = []
    for sublist in array:
        for item in sublist:
            temp.append(item)
    return np.array(temp)
  
def rmv(array,out): # removing galaxies in "out" from galaxies in "array"
    test_out =[] 
    for i in array:
        if i in out:
            test_out.append(False)
        else:
            test_out.append(True)
           
    return np.array(array)[test_out]


In [3]:
data = Table.read('data_all.txt', format = 'ascii')


In [4]:
data

index,ra,dec,redshift,Rgal,rabsmag,u_r,imc,aimc,cd,conx1,prmag,epetR,epetRc,iso_a,iso_b,abtrue,phi_iso_deg,vdisp,vdisperr,snmedian,rdev,ab_dev,rexp,ab_exp,mflag,mr,g_r,Ez,vflag,plate,MJD,fiberID,MPA_index,Mstar,BPTclass,flag3727,Z12logOH,Zerr,t3,N12logNH,NHerr,logNO,NOerr,sSFR,SFR,NUV_r,u_r_NSA,g_r_NSA,rabsmag_NSA,Mstar_NSA,Ne12logNeH,NeHerr,logNeO,NeOerr
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
749.0,38.049133,0.224026,0.054022,160.22,-20.101,2.755,1.0,1.0,-0.047,0.283,15.939,8.5,8.413,16.21,11.82,0.73,-80.52,223.27,6.29,33.68,2.434,0.807,1.536,0.833,0.0,16.139,0.952,160.223458454,9,407,51820,513,70459,10.55196475982666,5.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.68909740447998,-1.0694656372070312,5.886381035385732,2.5909545388571527,0.8840782662723994,-20.536251068115234,22597765120.0,nan,nan,nan,nan
750.0,38.352528,0.212491,0.053818,159.62,-19.256,2.565,1.0,1.0,-0.02,0.365,16.786,5.89,5.848,8.51,7.74,0.91,-8.39,67.03,5.59,21.47,2.485,0.887,1.788,0.827,0.0,16.837,0.871,159.625013119,9,407,51820,594,70517,10.169575691223145,3.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.156432151794434,-0.9351850748062134,inf,2.2251583317081285,0.7622379303778964,-19.708415985107422,10089863168.0,nan,nan,nan,nan
751.0,38.363598,0.210654,0.05405,160.31,-20.959,2.495,2.0,25.88,-0.297,0.417,15.09,13.78,13.704,25.21,10.3,0.41,-75.63,137.34,5.46,26.55,10.349,0.407,6.239,0.386,0.0,15.309,0.877,160.305594057,9,408,51821,351,70759,10.911660194396973,4.0,nan,nan,nan,nan,nan,nan,nan,nan,-10.782448768615723,0.17625018954277039,inf,2.1127458441288782,0.7120395549037929,-21.317277908325195,45481709568.0,nan,nan,nan,nan
761.0,54.936817,0.216794,0.201277,578.63,-21.711,2.88,1.0,1.0,0.034,0.323,17.361,5.02,4.906,8.83,7.48,0.85,-67.96,245.02,13.84,13.57,2.21,0.811,1.248,0.836,0.0,17.265,1.0,578.633830879,9,416,51811,472,78846,11.36347484588623,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.924013137817383,-0.517741858959198,nan,nan,nan,nan,nan,nan,nan,nan,nan
1009.0,54.534882,0.578615,0.128955,376.57,-20.604,2.767,1.0,1.0,-0.05,0.281,17.415,4.45,4.304,8.88,6.56,0.73,69.88,197.05,10.07,15.48,1.449,0.855,1.195,0.866,0.0,17.402,0.963,376.56649592,9,416,51811,385,78791,10.833581924438477,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.989643096923828,-1.1046427488327026,nan,nan,nan,nan,nan,nan,nan,nan,nan
1029.0,54.743076,0.508774,0.183435,529.41,-21.392,2.576,1.0,1.0,-0.026,0.313,17.453,4.11,3.951,8.53,7.38,0.86,76.4,232.49,13.07,13.31,1.669,1.0,0.988,0.984,0.0,17.427,0.994,529.408941519,9,416,51811,423,78820,11.235125541687012,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.931730270385742,-0.6697368025779724,nan,nan,nan,nan,nan,nan,nan,nan,nan
1037.0,54.867241,0.562636,0.139661,406.9,-20.776,2.936,1.0,1.0,-0.127,0.325,17.42,7.25,7.201,9.66,7.36,0.76,-1.13,173.33,14.65,11.04,2.552,0.633,1.299,0.664,0.0,17.464,0.925,406.901440565,9,416,51811,422,78819,10.913909912109375,2.0,nan,nan,nan,nan,nan,nan,nan,nan,-10.952052116394043,0.012539233081042767,nan,nan,nan,nan,nan,nan,nan,nan,nan
1041.0,54.856907,0.544031,0.139478,406.38,-20.961,2.775,1.0,1.0,-0.147,0.358,17.234,6.42,6.342,9.21,6.75,0.73,82.33,165.2,13.54,11.82,3.384,0.8,1.634,0.838,0.0,17.123,0.954,406.384142499,9,416,51811,428,78822,11.011194229125977,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,-11.529818534851074,-0.47579777240753174,nan,nan,nan,nan,nan,nan,nan,nan,nan
1048.0,54.977749,0.501912,0.139322,405.94,-20.943,2.223,2.0,25.78,-0.163,0.43,17.242,4.97,4.768,7.5,5.4,0.71,-10.86,133.73,10.1,13.41,3.845,0.68,2.005,0.704,0.0,17.385,0.749,405.943133443,9,416,51811,469,78845,10.933839797973633,2.0,nan,nan,nan,nan,nan,nan,nan,nan,-10.561537742614746,0.4192074835300

### V^2 classification

In [5]:
galzones = Table.read('DR7_galzones.dat', format = 'ascii') 
zonevoids = Table.read('DR7_zonevoids.dat', format = 'ascii')
total_glx = len(galzones)

In [6]:
print(total_glx)

707817


In [7]:
print(len(data))

707817


#### edge and outside galaxies

In [35]:
edge_bool = [bool(x) for x in np.array(np.array(galzones['edge']))] 
edge_gal = np.array(galzones['gal'][edge_bool])

In [36]:
out_bool = [bool(x) for x in np.array(np.array(galzones['out']))]
out_gal = np.array(galzones['gal'][out_bool])

#### void galaxies

In [37]:
void_zones = zonevoids[(zonevoids['void0'] != -1) & (zonevoids['void1'] != -1)]['zone'] # zones that are in voids

In [38]:
void_gal_ = []
for i in void_zones:
    void_gal_.append(list(galzones['gal'][galzones['zone'] == i]))

void_gal_flat = flatten(void_gal_) # around 200,000 are void galaxies

#### no outside galaxies within voids but  there are edge galaxies within voids

In [39]:
void_gal = rmv(void_gal_flat,edge_gal) # eliminating edge galazies, True for non edge, about 8000

In [40]:
void_gal = rmv(void_gal,out_gal)

#### wall galaxies

In [41]:
non_void_zones = zonevoids[(zonevoids['void0'] == -1)]['zone'] # zones that are not in voids

non_void_gal = []
for i in non_void_zones:
    non_void_gal.append(list(galzones['gal'][galzones['zone'] == i]))
        
wall_gal = rmv(flatten(non_void_gal),edge_gal)

In [42]:
wall_gal = rmv(wall_gal,out_gal)

### Check

In [43]:
len(wall_gal)+len(void_gal)+len(edge_gal)+len(out_gal), total_glx

(707817, 707817)

### Voidfinder classification

In [44]:
data_voidfinder = Table.read('ALL_vflag_comoving.txt', format = 'ascii')

In [45]:
wall_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 0)
void_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 1)

In [46]:
out_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 9)
edge_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 2)

### Check

In [47]:
len(out_gal_vf)+len(edge_gal_vf)+len(wall_gal_vf)+len(void_gal_vf) == len(data_voidfinder)

True

### add flag columns and save file

In [48]:
flag_vsquared=[] # checks all galaxies in each statement as a double check for any overlaps
for i in range(len(data)):
    if i in wall_gal:
        flag_vsquared.append(0)
    if i in void_gal:
        flag_vsquared.append(1)
    if i in edge_gal:
        flag_vsquared.append(2)
    if i in out_gal:
        flag_vsquared.append(9)
        
flag_vsquared_vf=[]
for i in range(len(data)):
    if i in wall_gal_vf:
        flag_vsquared_vf.append(0)
    if i in void_gal_vf:
        flag_vsquared_vf.append(1)
    if i in edge_gal_vf:
        flag_vsquared_vf.append(2)
    if i in out_gal_vf:
        flag_vsquared_vf.append(9)
        
print(len(flag_vsquared),len(flag_vsquared_vf), len(data)) #check



707817 707817 707817


In [49]:
flags_table = Table([flag_vsquared_vf,flag_vsquared], names=('flag_voidfinder', 'flag_vsquared'))

data.add_columns(flags_table.columns.values())

ascii.write(data, 'data_flags_updated_Aug11.dat')  